In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data63881


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [3]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
# !pip install beautifulsoup4 -t /home/aistudio/external-libraries
!pip install lightgbm==2.3.1 -t /home/aistudio/external-libraries
# !pip install catboost==0.23 -t /home/aistudio/external-libraries
# !pip install jieba -t /home/aistudio/external-libraries
# !pip install gensim -t /home/aistudio/external-libraries
!pip uninstall --yes pandas
!pip install pandas==1.0.5 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 1.2MB 15.5MB/s eta 0:00:01
     |████████████████████████████████| 25.9MB 9.9MB/s eta 0:00:012
     |████████████████████████████████| 14.5MB 8.6MB/s eta 0:00:011
     |████████████████████████████████| 6.8MB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 307kB 23.8MB/s eta 0:00:01
ERROR: seaborn 0.10.0 requires pandas>=0.22.0, which is not installed.
ERROR: paddlehub 1.6.0 requires pandas; python_version >= "3", which is not installed.
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 10.1MB 22.3MB/s eta 0:00:01
     |████████████████████████████████| 512kB 15.3MB/s eta 0:00:01
     |████████████████████████████████| 235kB 23.3MB/s eta 0:00:01
     |████████████████████████████████| 14.5MB 10.1MB/s eta 0:00:01    |█████▏                          | 2

In [4]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [5]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [6]:
train_transaction = pd.read_csv('data/data63881/train_transaction.csv')
test_transaction = pd.read_csv('data/data63881/test_transaction.csv')
train_identity = pd.read_csv('data/data63881/train_identity.csv')
test_identity = pd.read_csv('data/data63881/test_identity.csv')

In [7]:
train = train_transaction.merge(train_identity, on='TransactionID', how='left')
test = test_transaction.merge(test_identity, on='TransactionID', how='left')
data = pd.concat([train, test], axis=0, ignore_index=True)
del train, test
gc.collect()

0

In [8]:
object_cols = ['ProductCD', 'card4', 'card6', 'DeviceType', 'DeviceInfo', 'P_emaildomain', 'R_emaildomain']
M_cols = ['M{}'.format(i) for i in range(1, 10)]
id_cols = ['id_12', 'id_16', 'id_27', 'id_28', 'id_29', 'id_35', 'id_36', 'id_37', 'id_38', 'id_15',
           'id_23', 'id_34', 'id_30', 'id_31', 'id_33']
cat_cols = object_cols + M_cols + id_cols

In [ ]:
for i in cat_cols:
    le = LabelEncoder()
    data[i] = le.fit_transform(data[i].astype(str))
    data[i] = data[i].astype('category')

In [ ]:
train = data[data['isFraud'].notnull()]
test = data[data['isFraud'].isnull()]

y = train['isFraud']
train.drop(['isFraud', 'TransactionID'], axis=1, inplace=True)
test.drop(['isFraud', 'TransactionID'], axis=1, inplace=True)
used_cols = train.columns
test = test[used_cols]

X_train, X_valid, y_train, y_valid = train_test_split(train, y, random_state=2020)

In [ ]:
# def auc_select(X_train, y_train, X_valid, y_valid, cols, threshold=0.52):
#     useful_dict = dict()
#     useless_dict = dict()
#     params = {
#         'objective': 'binary',
#         'boosting': 'gbdt',
#         'metric': 'auc',
#         'learning_rate': 0.1,
#         'num_leaves': 31,
#         'lambda_l1': 0,
#         'lambda_l2': 1,
#         'num_threads': 23,
#         'min_data_in_leaf': 20,
#         'first_metric_only': True,
#         'is_unbalance': True,
#         'max_depth': -1,
#         'seed': 2020
#     }
#     for i in cols:
#         print(i)
#         try:
#             lgb_train = lgb.Dataset(X_train[[i]].values, y_train)
#             lgb_valid = lgb.Dataset(X_valid[[i]].values, y_valid, reference=lgb_train)
#             lgb_model = lgb.train(
#                 params,
#                 lgb_train,
#                 valid_sets=[lgb_valid, lgb_train],
#                 num_boost_round=1000,
#                 early_stopping_rounds=50,
#                 verbose_eval=500
#             )
#             print('*' * 10)
#             print(lgb_model.best_score['valid_0']['auc'])
#             if lgb_model.best_score['valid_0']['auc'] > threshold:
#                 useful_dict[i] = lgb_model.best_score['valid_0']['auc']
#             else:
#                 useless_dict[i] = lgb_model.best_score['valid_0']['auc']
#         except:
#             print('Error: ', i)
#     useful_cols = list(useful_dict.keys())
#     useless_cols = list(useless_dict.keys())
#     return useful_dict, useless_dict, useful_cols, useless_cols


# useful_dict, useless_dict, useful_cols, useless_cols = auc_select(X_train, y_train, X_valid, y_valid, used_cols, threshold=0.52)
# print(useless_cols)

In [ ]:
auc_to_drop = ['card4', 'C3', 'V41', 'V98', 'V100', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V113', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V135', 'V138', 'V141', 'V142', 'V161', 'V162', 'V163', 'V286', 'V297', 'V299', 'V300', 'V301', 'V305', 'V311', 'V319', 'V325', 'V326', 'V334', 'id_07', 'id_08', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_32', 'id_34']

X_train.drop(auc_to_drop, axis=1, inplace=True)
X_valid.drop(auc_to_drop, axis=1, inplace=True)
train.drop(auc_to_drop, axis=1, inplace=True)
test.drop(auc_to_drop, axis=1, inplace=True)
gc.collect()

40

In [ ]:
%%time
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid, reference=dtrain)
params = {
    'objective': 'binary',
    'boosting': 'gbdt',
    'metric': 'auc',
    # 'metric': 'None',  # 用自定义评估函数是将metric设置为'None'
    'learning_rate': 0.1,
    'num_leaves': 31,
    'lambda_l1': 0,
    'lambda_l2': 1,
    'num_threads': 23,
    'min_data_in_leaf': 20,
    'first_metric_only': True,
    'is_unbalance': True,
    'max_depth': -1,
    'seed': 2020
}
valid_model = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dvalid],
    num_boost_round=1000000,
    early_stopping_rounds=200,
    verbose_eval=300
)

importance = valid_model.feature_importance(importance_type='gain')
feature_name = valid_model.feature_name()
df_importance = pd.DataFrame({
    'feature_name': feature_name,
    'importance': importance
}).sort_values(by='importance', ascending=False)
df_importance['normalized_importance'] = df_importance['importance'] / df_importance['importance'].sum()
df_importance['cumulative_importance'] = np.cumsum(df_importance['normalized_importance'])
record_low_importance = df_importance[df_importance['cumulative_importance'] > 0.90]
to_drop = list(record_low_importance['feature_name'])
print(to_drop)
df_importance.to_csv('sub/imp.csv', index=False)

Training until validation scores don't improve for 200 rounds
[600]	training's auc: 0.994288	valid_1's auc: 0.955613
[900]	training's auc: 0.997729	valid_1's auc: 0.959101
[1200]	training's auc: 0.999054	valid_1's auc: 0.960196
[1500]	training's auc: 0.99961	valid_1's auc: 0.960724
[1800]	training's auc: 0.999843	valid_1's auc: 0.961338
[2100]	training's auc: 0.999939	valid_1's auc: 0.962051
[2400]	training's auc: 0.999977	valid_1's auc: 0.962236
[2700]	training's auc: 0.999991	valid_1's auc: 0.962747
[3000]	training's auc: 0.999997	valid_1's auc: 0.963041
[3300]	training's auc: 0.999999	valid_1's auc: 0.96321
Early stopping, best iteration is:
[3154]	training's auc: 0.999999	valid_1's auc: 0.963246
Evaluated only: auc
['V54', 'V76', 'M3', 'V285', 'M8', 'V62', 'V243', 'V75', 'C12', 'V61', 'V36', 'V53', 'M7', 'M9', 'id_06', 'V56', 'id_05', 'V127', 'V13', 'V128', 'V12', 'V55', 'V78', 'V189', 'V131', 'id_14', 'V296', 'V20', 'V35', 'V67', 'V130', 'id_13', 'V90', 'V38', 'V308', 'V281', 'D14

In [ ]:
X_train.drop(to_drop, axis=1, inplace=True)
X_valid.drop(to_drop, axis=1, inplace=True)
train.drop(to_drop, axis=1, inplace=True)
test.drop(to_drop, axis=1, inplace=True)
gc.collect()

41

In [ ]:
%%time
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid, reference=dtrain)
dall = lgb.Dataset(train, y)

valid_model2 = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dvalid],
    num_boost_round=1000000,
    early_stopping_rounds=200,
    verbose_eval=300
)

Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.981882	valid_1's auc: 0.947377
[600]	training's auc: 0.993437	valid_1's auc: 0.955178
[900]	training's auc: 0.997105	valid_1's auc: 0.957588
[1200]	training's auc: 0.998756	valid_1's auc: 0.958928
[1500]	training's auc: 0.99946	valid_1's auc: 0.959922
[1800]	training's auc: 0.99978	valid_1's auc: 0.960495
[2100]	training's auc: 0.9999	valid_1's auc: 0.960727
[2400]	training's auc: 0.999957	valid_1's auc: 0.96103
[2700]	training's auc: 0.999981	valid_1's auc: 0.961271
[3000]	training's auc: 0.999993	valid_1's auc: 0.961598
[3300]	training's auc: 0.999998	valid_1's auc: 0.961953
[3600]	training's auc: 0.999999	valid_1's auc: 0.962003
[3900]	training's auc: 1	valid_1's auc: 0.962268
[4200]	training's auc: 1	valid_1's auc: 0.962444
[4500]	training's auc: 1	valid_1's auc: 0.962706
[4800]	training's auc: 1	valid_1's auc: 0.963025
[5100]	training's auc: 1	valid_1's auc: 0.963088
[5400]	training's auc: 1	vali

In [ ]:
%%time
model = lgb.train(
    params,
    dall,
    num_boost_round=valid_model2.best_iteration
)

CPU times: user 46min 33s, sys: 5.33 s, total: 46min 38s
Wall time: 11min 43s


In [ ]:
%%time
pred = model.predict(test)
sub = pd.DataFrame({'id': range(len(test))})
sub['isFraud'] = pred
sub.to_csv('./sub/baseline_{}_02.csv'.format(time.strftime('%Y%m%d')), index=False, header=None)

CPU times: user 1min 13s, sys: 68 ms, total: 1min 13s
Wall time: 19 s
